# Generating images

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter, FuncFormatter
from pathlib import Path
from PIL import Image
import io

### Pandas options

In [ ]:
pd.set_option('display.max_row', None)
pd.set_option('display.max_column', None)

## File paths

In [ ]:
root = Path()
info_path = root / 'info'
data_proc = root / 'data' / 'data_proc'
out_path =root / 'output'
out_path.mkdir(exist_ok=True)
out_tables = out_path / 'exracted_tables'
out_tables.mkdir(exist_ok=True)
stats_subj_file_name = out_tables / 'stats_full_subj_protocol.pkl'
stats_prot_file_name = out_tables / 'stats_full_exp_protocol.pkl'
out_path.mkdir(exist_ok=True)
save_fig_path = out_path / 'figs'
diff_fig_path = save_fig_path / 'diff'
ir_diff_fig_path = save_fig_path / 'ir_diff'
mse_fig_path = save_fig_path / 'mse'
save_fig_path.mkdir(exist_ok=True)
diff_fig_path.mkdir(exist_ok=True)
ir_diff_fig_path.mkdir(exist_ok=True)
mse_fig_path.mkdir(exist_ok=True)

## Reading Global data

In [ ]:
df_stats_global = pd.read_pickle(stats_prot_file_name)
df_stats_global.head()

## Reading per Subject Data

In [ ]:
df_stats = pd.read_pickle(stats_subj_file_name)
df_stats.head()


## Difference tables

In [ ]:
# difference tables
df_diff = df_stats.subtract(df_stats_global)
df_diff = df_diff.reorder_levels(['Subject', 'Activity', 'Type', 'Channel', 'stats'], axis=1)
df_diff.sort_index(axis=1, level=[0,1,2,3], inplace=True)


In [ ]:
# save files
file_name = out_tables/'diff_mean_subject.xlsx'
df_diff.to_excel(file_name)
file_name = out_tables/'diff_mean_subject.pkl'
df_diff.to_pickle(file_name)
# head
df_diff.head()

## Generating images for all the subjects

In [ ]:
# image options
color = ['royalblue',  'gold', 'firebrick', 'seagreen']
marks = ['o',  's', 'D', '^' ]
mrk_size = [1.5, 1, 1 ,1.1]
use_title = True


# log ticks formatting
formatter_x = ScalarFormatter()
formatter_x.set_scientific(False)

# Configure or save image options
img_config = not True
save_eps = True
max_display_img_config = 8

stat_name = 'mean'

col_lev_list = df_stats.columns.levels

subj_list = col_lev_list[0]

exp_protocol_list = pd.MultiIndex.from_product( df_stats.columns.levels[1:3])

channels = col_lev_list[3]

if img_config ==  True:
    subj_list = subj_list[:max_display_img_config]

# for each subject
for subj_code in subj_list:
    # create plot
    fig,ax = plt.subplots(2,2, figsize=(18,9))

    if use_title == True:
        fig.suptitle('Subject code: {} (difference from mean value)'.format(subj_code ), fontsize=35, y=1)

    fig.supylabel('$DMC_{x}^{2}$', fontsize=25, x=0.05)
    fig.supxlabel('Time window size (n)', fontsize=25)

    min_y = -1.1
    max_y = 1.1

    for i_ex, exp_protocol in enumerate(exp_protocol_list):

        # slice dataframes
        df_aux = df_diff.loc[:, ([subj_code], [exp_protocol[0]], [exp_protocol[1]], slice(None), [stat_name] ) ]
  
        # for each experiment
    
        fig_row = i_ex//2
        fig_col = i_ex%2

        ax[fig_row][fig_col].set_title(" - ".join(exp_protocol) , fontsize=15)

        ax[fig_row][fig_col].set_ylim([min_y, max_y])

        ax[fig_row][fig_col].set_facecolor("#dfdfdf")
        #ax[fig_row][fig_col].loglog()
        ax[fig_row][fig_col].semilogx()
    
        ax[fig_row][fig_col].xaxis.set_major_formatter(formatter_x)
        ax[fig_row][fig_col].set_yticks(np.arange(-1,1.1,0.25))
        ax[fig_row][fig_col].grid(visible=True, which='both', axis='both',  linestyle='dashdot', linewidth=.5)

        # for each channel
        for i_ch , chan in enumerate(channels):

            ax[fig_row][fig_col].plot(df_aux.index.values, df_aux.loc[:, (slice(None), slice(None), slice(None),[chan]) ], c= color[i_ch], marker=marks[i_ch], label = chan, markersize=6* mrk_size[i_ch],  linestyle=':', alpha=.8 ,linewidth=1.5)

        # janela temporal da figura
        ax[fig_row][fig_col].axvline(x=65.6, color='indigo', linestyle='dashed',alpha=.50,linewidth=2.5)

    # Legenda
    ax[1,1].legend( title = "Y variable\nchannel:", title_fontsize=12, bbox_to_anchor=(1,1.45), loc='upper left',  prop={'size': 15})

    # config or save images
    if img_config == True:
        fig.show()
    else:
        fig_file_name = diff_fig_path / (subj_code + '.jpg')
        fig.savefig(fig_file_name)
        # Save the image in memory in PNG format
        if save_eps == True:
            fig_file_name = fig_file_name.with_suffix('.eps')
            with  io.BytesIO() as png1:
                fig.savefig(png1, format="png")
                # Load this image into PIL
                with Image.open(png1) as png2:
                # Save as eps
                    png2 = png2.convert('RGB')
                    png2.save(fig_file_name, lossless = True)

        # Close images
        plt.close()
        fig.clf()

        print(subj_code, end="...")
print('END')       

# Real - Imaginary images

In [ ]:
df_real = df_stats.loc[:, (slice(None), slice(None), ["Real"], slice(None), slice(None) ) ]
df_real = df_real.droplevel(2, axis=1)
df_real.head()


In [ ]:
df_img = df_stats.loc[:, (slice(None), slice(None), ["Imaginary"], slice(None), slice(None) ) ]
df_img = df_img.droplevel(2, axis=1)
df_img.head()

In [ ]:
df_diff_ri = df_img.subtract(df_real)
df_diff_ri.head()

In [ ]:
# save files
file_name = out_tables/'diff_real_img_subject.xlsx'
df_diff_ri.to_excel(file_name)
file_name = out_tables/'diff_real_img_subject.pkl'
df_diff_ri.to_pickle(file_name)

In [ ]:
# image options
color = ['royalblue',  'gold', 'firebrick', 'seagreen']
marks = ['o',  's', 'D', '^' ]
mrk_size = [1.5, 1, 1 ,1.1]
use_title = True


# log ticks formatting
formatter_x = ScalarFormatter()
formatter_x.set_scientific(False)

# Configure or save image options
img_config = not True
save_eps = True
max_display_img_config = 5

stat_name = 'mean'

col_lev_list = df_diff_ri.columns.levels

subj_list = col_lev_list[0]

act_list = df_stats.columns.levels[1]

channels = col_lev_list[2]

if img_config ==  True:
    subj_list = subj_list[:max_display_img_config]

# for each subject
for subj_code in subj_list:
    # create plot
    fig,ax = plt.subplots(1,2, figsize=(18,5))
    fig.subplots_adjust( top=.8)
   


    if use_title == True:
        fig.suptitle('Subject code: {} difference (Imaginary - real)'.format(subj_code), fontsize=35,y=1)
        
    fig.supylabel('$DMC_{x}^{2}$', fontsize=25, x=0.05)
    fig.supxlabel('Time window size (n)', fontsize=25)

    min_y = -1
    max_y = 1.1

    for i_act, act in enumerate(act_list):

        # slice dataframes
        df_aux = df_diff_ri.loc[:, ([subj_code], [act], slice(None), [stat_name] ) ]
  
        # for each experiment
    
        fig_num = i_act
  

        ax[fig_num].set_title( act , fontsize=15)

        ax[fig_num].set_ylim([min_y, max_y])

        ax[fig_num].set_facecolor("#dfdfdf")

        ax[fig_num].semilogx()
    
        ax[fig_num].xaxis.set_major_formatter(formatter_x)
        ax[fig_num].set_yticks(np.arange(min_y, max_y, 0.25))
        ax[fig_num].grid(visible=True, which='both', axis='both',  linestyle='dashdot', linewidth=.5)

        # for each channel
        for i_ch , chan in enumerate(channels):

            ax[fig_num].plot(df_aux.index.values, df_aux.loc[:, (slice(None),  slice(None),[chan], slice(None)) ], c= color[i_ch], marker=marks[i_ch], label = chan, markersize=6* mrk_size[i_ch],  linestyle=':', alpha=.8 ,linewidth=1.5)

        # janela temporal da figura
        ax[fig_num].axvline(x=65.6, color='indigo', linestyle='dashed',alpha=.50,linewidth=2.5)

    # Legenda
    ax[1].legend( title = "Y variable\nchannel:", title_fontsize=12, bbox_to_anchor=(1,.75), loc='upper left',  prop={'size': 15})

    # config or save images
    if img_config == True:
        fig.show()
    else:
        fig_file_name = ir_diff_fig_path / (subj_code + '.jpg')
        fig.savefig(fig_file_name)
        # Save the image in memory in PNG format
        if save_eps == True:
            fig_file_name = fig_file_name.with_suffix('.eps')
            with io.BytesIO() as png1:
                fig.savefig(png1, format="png")
                # Load this image into PIL
                with Image.open(png1) as png2:
                # Save as eps
                    png2 = png2.convert('RGB')
                    png2.save(fig_file_name, lossless = True)
            

        # Close images
        plt.close()
        fig.clf()
        print(subj_code, end="...")
print('END')       

## Error bar

In [ ]:
df_mse = pd.DataFrame(df_diff.pow(2).mean())
df_mse.head()

In [ ]:
df_mse_mean = df_mse.loc[ (slice(None), slice(None), slice(None), slice(None), ['mean']), :]
df_mse_mean = df_mse_mean.droplevel(4, axis=0)

df_mse_mean.head(10)

In [ ]:
channels_list = df_mse_mean.index.levels[3]
activity_list =  df_mse_mean.index.levels[1].to_list()[::-1]
type_list =  df_mse_mean.index.levels[2].to_list()[::-1]

color_list = ['teal', 'indigo', 'cornflowerblue', 'steelblue']

img_config = not True
save_eps = True
max_display_img_config = 5

use_title = True

if img_config ==  True:
    subj_list = subj_list[:max_display_img_config]

for chan in channels_list:

    fig,ax = plt.subplots(4,1, figsize=(20,10))

    if use_title == True:
        fig.suptitle('MSE: Channel {}'.format(chan), fontsize=30,y=1)

    i_aux = 0
    for i_act, str_act in enumerate(activity_list):
        
        for i_type, str_type in enumerate(type_list):

            i_ax = i_act + i_type + i_aux

            df_act_type = df_mse_mean.loc[ (slice(None), [str_act], [str_type], [chan]), :]
            ax[i_ax].set_facecolor("#dfdfdf")
            ax[i_ax].grid(visible=True, which='both', axis='both',  linestyle='dashdot', linewidth=.5)

            ax[i_ax].bar(["-".join(i) for i in df_act_type.index], df_act_type.values.flatten(), color = color_list[i_ax])
            ax[i_ax].set_yticks(np.arange(0,.51,0.1))

            ax[i_ax].set_ylabel(' - '.join([str_act, str_type]))

            if i_ax < 3:
                ax[i_ax].set_xticklabels([])
            else:
 
                ax[i_ax].set_xticklabels([i[0] for i in df_act_type.index], rotation=90, fontsize='small')
        i_aux = i_aux + 1
    
    # Save figs
    if img_config == True:
        fig.show()
    else:
        fig_file_name = mse_fig_path / ('mse' + chan + '.jpg')
        fig.savefig(fig_file_name)
        # Save the image in memory in PNG format
        if save_eps == True:
            fig_file_name = fig_file_name.with_suffix('.eps')
            with io.BytesIO() as png1:
                fig.savefig(png1, format="png")L
                # Load this image into PIL
                with Image.open(png1) as png2:
                # Save as eps
                    png2 = png2.convert('RGB')
                    png2.save(fig_file_name, lossless = True)
        # Close images
        plt.close()
        fig.clf()
        print(chan, end="...")
print('END')


In [ ]:
df_mse_table = df_mse_mean.unstack(level=(2,3))
df_mse_table = df_mse_table.droplevel(0, axis=1)
df_mse_table.head()

In [ ]:
# save files
file_name = out_tables/'mse_table.xlsx'
df_mse_table.to_excel(file_name)
file_name = out_tables/'mse_table.pkl'
df_mse_table.to_pickle(file_name)

In [ ]:
df_mse_table.max()

## Images interval

In [ ]:
interval = 6.25 * 10**-3
print(interval)
print(100 * interval)
print(0.41 / interval)